In [1]:
import sys, os
from pathlib import Path
#親ディレクトリのパスを取得,これはJupyter系特有の書き方
current_dir = os.path.join(Path().resolve())
# モジュールのあるパスを追加
sys.path.append(str(current_dir) + '/../')
import carats_csv_to_pickle as cctp

import pandas as pd

date = '20190818'
source_time = '00_12'

target_airport = 'RJTT'  # 目的空港のICAOコードを指定

trk_frame = pd.read_csv(f'~/Desktop/201908/trk{date}_{source_time}.csv')
trk_frame.columns = ["time", "Callsign", "Latitude", "Longitude", "Altitude", "Type"]
ap_frame = pd.read_csv("~/programing/CARATS/MakeApt_v1/Aerodrome_utf8.txt",
                       header=None, usecols=[0, 2, 3], delim_whitespace=True, names=["ICAO", "Latitude", "Longitude"])

ap_frame["Latitude_decimal"] =ap_frame["Latitude"].apply(lambda x: round((int(x[:2]) + int(x[2:4])/60 + int(x[4:6])/3600),5))
ap_frame["Longitude_decimal"] =ap_frame["Longitude"].apply(lambda x: round((int(x[:3]) + int(x[3:5])/60 + int(x[5:7])/3600),5))

In [2]:
trk_frame

,time,Callsign,Latitude,Longitude,Altitude,Type
0,00:00:00.0,AG28114,46.433333,157.133333,32000,B748
1,00:00:00.0,AG28158,36.716667,163.783333,37000,A333
2,00:00:00.0,AG28210,44.566667,153.966667,34000,B748
3,00:00:00.0,AG28256,43.066667,151.700000,33000,B77W
4,00:00:00.0,AG28295,42.333333,150.650000,32000,B77W
...,...,...,...,...,...,...
833304,11:59:59.9,AG30607,27.090998,127.631977,23760,B738
833305,11:59:59.9,AG30615,41.873623,140.380254,10100,SF34
833306,11:59:59.9,AG30641,38.002868,140.600796,10853,CRJ7
833307,11:59:59.9,AG30645,31.858270,130.594901,6893,SF34


In [3]:
filterd_trk_frame = trk_frame
# filterd_trk_frameをCallsign, timeの順でソートする
filterd_trk_frame = filterd_trk_frame.sort_values(by=['Callsign', 'time'], ascending=[True, True])
first_trk = filterd_trk_frame.groupby("Callsign").first()

In [4]:
# 各Callsignについて、最初のデータと最後のデータを取得する
first_data = filterd_trk_frame.groupby('Callsign').first()
last_data = filterd_trk_frame.groupby('Callsign').last()
print(first_data)
print(last_data)

                time   Latitude   Longitude  Altitude  Type
Callsign                                                   
AG28070   00:01:00.0  47.133333  158.416667     32000  B77W
AG28114   00:00:00.0  46.433333  157.133333     32000  B748
AG28158   00:00:00.0  36.716667  163.783333     37000  A333
AG28161   00:04:37.9  32.502848  126.921321     30000  B77L
AG28210   00:00:00.0  44.566667  153.966667     34000  B748
...              ...        ...         ...       ...   ...
AG30657   11:59:00.9  39.618765  140.182457      1500  B738
AG30659   11:59:10.9  37.667913  133.004152     34997  A321
AG30660   11:59:13.4  33.843872  132.669225      1400  B738
AG30662   11:59:53.9  42.789237  141.686291      1200  E170
AG30663   11:59:57.4  34.408734  135.214024      1375  B738

[2151 rows x 5 columns]
                time   Latitude   Longitude  Altitude  Type
Callsign                                                   
AG28070   00:14:45.0  48.250000  160.600000     32000  B77W
AG28114   00:22

In [5]:
# last_dataが2000ft以下のものを抽出する
landed_flights = last_data[last_data['Altitude'] <= 2000]
landed_flights

,time,Latitude,Longitude,Altitude,Type
Callsign,,,,,
AG28443,00:36:51.6,35.546659,139.828813,249,A21N
AG28460,00:04:32.5,35.546743,139.828749,240,A321
AG28471,01:14:13.4,34.424844,135.240954,200,B789
AG28482,00:54:03.6,35.545642,139.827625,238,B77W
AG28526,00:52:35.6,35.547295,139.829519,245,A320
...,...,...,...,...,...
AG30431,11:56:13.6,33.600017,130.441750,132,DH8B
AG30451,11:48:27.9,30.610211,130.984207,903,AT46
AG30468,11:59:59.9,43.086904,144.167833,1301,SF34


In [6]:
# landed_flightsの緯度経度から、RJCCに着陸した航空機を抽出する。
# 使用するデータはap_frameの緯度経度。
# 出力するdataframeは、[Callsign, Altitude, Latitude, Longitude, Distance_to_Airport]

# RJCCの緯度経度を取得
target_airport_df = ap_frame[ap_frame['ICAO'] == target_airport]
target_airport_lat = target_airport_df['Latitude_decimal'].values[0]
target_airport_lon = target_airport_df['Longitude_decimal'].values[0]
# landed_flightsにDistance_to_Airport列を追加
import numpy as np
landed_flights['Distance_to_Airport'] = np.sqrt((landed_flights['Latitude'] - target_airport_lat)**2 + (landed_flights['Longitude'] - target_airport_lon)**2) * 111.32
# Distance_to_Airportが10km以下のものを抽出
landed_to_target_airport = landed_flights[landed_flights['Distance_to_Airport'] <= 10.0]
landed_to_target_airport[['Altitude', 'Latitude', 'Longitude', 'Distance_to_Airport']]

/var/folders/qx/20wfqtsj4691p3qc2hdnwwxw0000gn/T/ipykernel_10049/1607459571.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  landed_flights['Distance_to_Airport'] = np.sqrt((landed_flights['Latitude'] - target_airport_lat)**2 + (landed_flights['Longitude'] - target_airport_lon)**2) * 111.32


,Altitude,Latitude,Longitude,Distance_to_Airport
Callsign,,,,
AG28443,249,35.546659,139.828813,5.361972
AG28460,240,35.546743,139.828749,5.353627
AG28482,238,35.545642,139.827625,5.248299
AG28526,245,35.547295,139.829519,5.430605
AG28597,281,35.548723,139.831122,5.590907
...,...,...,...,...
AG30172,221,35.529247,139.790836,2.891291
AG30181,274,35.529220,139.790822,2.893496
AG30193,825,35.499220,139.812117,6.942412


In [7]:
# trk_frameからlanded_to_rjccのCallsignに該当するデータを抽出する
# 抽出した後、北緯39度以北のデータのみ抽出する。
landed_trk_frame = trk_frame[trk_frame['Callsign'].isin(landed_to_target_airport.index)]
landed_trk_frame = landed_trk_frame[landed_trk_frame['Latitude'] >= target_airport_df['Latitude_decimal'].values[0] - 10.0]
landed_trk_frame = landed_trk_frame[landed_trk_frame['Latitude'] <= target_airport_df['Latitude_decimal'].values[0] + 10.0]
landed_trk_frame = landed_trk_frame[landed_trk_frame['Longitude'] >= target_airport_df['Longitude_decimal'].values[0] - 10.0]
landed_trk_frame = landed_trk_frame[landed_trk_frame['Longitude'] <= target_airport_df['Longitude_decimal'].values[0] + 10.0]
landed_trk_frame

# landed_trk_frameをCallsign, timeの順でソートする
# ソートしたものを、3分の１のデータのみ抽出する
landed_trk_frame = landed_trk_frame.sort_values(by=['Callsign', 'time'], ascending=[True, True])
landed_trk_frame = landed_trk_frame.groupby('Callsign').apply(lambda x: x.iloc[::3, :]).reset_index(drop=True)
landed_trk_frame


,time,Callsign,Latitude,Longitude,Altitude,Type
0,00:00:06.8,AG28443,33.013873,137.064984,35000,A21N
1,00:00:35.3,AG28443,33.050839,137.123843,35000,A21N
2,00:01:03.8,AG28443,33.089114,137.183962,34975,A21N
3,00:01:32.3,AG28443,33.126821,137.243523,34975,A21N
4,00:02:00.8,AG28443,33.163567,137.302076,34969,A21N
...,...,...,...,...,...,...
27522,11:57:54.7,AG30401,35.477976,139.827144,1252,A21N
27523,11:58:18.6,AG30401,35.491215,139.817843,1031,A21N
27524,11:58:42.6,AG30401,35.505035,139.808038,724,A21N
27525,11:59:06.6,AG30401,35.518735,139.798301,425,A21N


In [8]:

# landed_trk_frameのLatitude, Longitudeすべてをgeojson形式で出力する

# 出力形式は以下の通り
# {
#   "type": "FeatureCollection",
#   "features": [
#     {
#       "type": "Feature",
#       "properties": {
#         "_markerType": "CircleMarker",
#         "_color": "#000000",
#         "_opacity": 0.5,
#         "_weight": 3,
#         "_fillColor": "#ff0000",
#         "_fillOpacity": 0.5,
#         "_radius": 2
#       },
#       "geometry": {
#         "type": "Point",
#         "coordinates": [
#           140.449219,
#           42.358544
#         ]
#       },
#       "geometry": {
#         "type": "Point",
#         "coordinates": [
#           140.394287,
#           42.240719
#         ]
#       }
#     }
#   ]
# }
geojson = {
    "type": "FeatureCollection",
    "features": []
}
for index, row in landed_trk_frame.iterrows():
    feature = {
        "type": "Feature",
        "properties": {
            "_markerType": "CircleMarker",
            "_color": "#000000",
            "_opacity": 0.5,
            "_weight": 3,
            "_fillColor": "#ff0000",
            "_fillOpacity": 0.5,
            "_radius": 2
        },
        "geometry": {
            "type": "Point",
            "coordinates": [
                row['Longitude'],
                row['Latitude']
            ]
        }
    }
    geojson['features'].append(feature)
import json
with open(f'landed_to_{target_airport}_{date}_{source_time}.geojson', 'w') as f:
    json.dump(geojson, f, indent=4)